## Задача 1. Матричные и векторные нормы. Обусловленность систем линейных алгебраических уравнений (СЛАУ).

### Задание первое: решить две системы (одну с “точной” правой частью, вторую с изменённой), найти число обусловленности исходной матрицы, посчитать фактическую погрешность и оценку погрешности (задание - 10.4, варианты - 10.5 по методичке, стр.90). Затем реализовать три метода: Гаусса с выбором главного элемента (по желанию) для решения СЛАУ, метод Жордана для нахождения обратной матрицы и метод LU-разложения для нахождения определителя матрицы (варианты - 11.6 по методичке, стр.94).
### Вариант 7
### Первая часть

$
A =
\begin{pmatrix}
    -402.90 & 200.7 \\
    1204.20 & -603.6
\end{pmatrix}
$

Входные данные

In [1]:
import numpy as np

A = np.array([
    [-402.90, 200.7],
    [1204.20, -603.6]
], dtype=np.float64)

b = np.array([
    [200],
    [-600]
], dtype=np.float64)

b_other = np.array([
    [199],
    [-601]
], dtype=np.float64)

Conditional numbers:

In [2]:
cond_A = np.linalg.cond(A, p='fro')
print(f"Cond(A) = {cond_A}")


Cond(A) = 1338.0029850746614


Solve $Ax = b$

In [3]:
x = np.linalg.solve(A, b)
x

array([[-0.19900498],
       [ 0.59701493]])

Solve $A\overline{x} = \overline{b}$

In [4]:
x_other = np.linalg.solve(A, b_other)
x_other

array([[0.33452736],
       [1.66308458]])

Relative actual error $\delta x = \|\overline{x} - x\| / \|x\|$

In [5]:
relative_actual_error = np.linalg.norm(x - x_other, ord='fro') / np.linalg.norm(x, ord='fro')
print(f"Relative actual error = {relative_actual_error}")

Relative actual error = 1.8943391987709555


Error estimate

In [6]:
error_estimate = cond_A * (np.linalg.norm(b - b_other, ord='fro') / np.linalg.norm(b, ord='fro'))
print(f"Error estimate = {error_estimate}")

Error estimate = 2.9918656287245793


### Вторая часть

$
A =
\begin{pmatrix}
    9.331343 & 1.120045 & -2.880925 \\
    1.120045 & 7.086042 & 0.670297 \\
    -2.880925 & 0.670297 & 5.622534
\end{pmatrix}
$

$
b =
\begin{pmatrix}
    7.570463 \\
    8.876384 \\
    3.411906
\end{pmatrix}
$

Входные данные

In [7]:
A = np.array([
    [9.331343, 1.120045, -2.880925],
    [1.120045, 7.086042, 0.670297],
    [-2.880925, 0.670297, 5.622534]
], dtype=np.float64)

b = np.array([7.570463, 8.876384, 3.411906], dtype=np.float64).reshape([3, 1])

Функция выбора главного элемента. Возвращает перестановки для $x$

In [8]:
def choose_main_element(A: np.ndarray) -> np.ndarray:
    n = A.shape[0]

    x_permutations = np.arange(0, n)
    for k in range(n):
        temp_matrix = A[k:n, k:n]
        i, j = np.unravel_index(np.abs(temp_matrix).argmax(), temp_matrix.shape)
        i += k
        j += k
        A[[k, i]] = A[[i, k]]
        A[:, [k, j]] = A[:, [j, k]]

        temp = x_permutations[k]
        x_permutations[k] = x_permutations[j]
        x_permutations[j] = temp

    return x_permutations

Метод Гаусса

In [9]:
def gaussian_method(A: np.ndarray, b: np.ndarray) -> np.ndarray:
    concat_A = np.append(A, b, axis=1)
    x_permutations = choose_main_element(concat_A)

    n = A.shape[0]
    # Forward
    for k in range(n):
        tmp = concat_A[k, k]
        if (tmp != 0):
            for j in range(k, n + 1):
                concat_A[k, j] /= tmp

        for i in range(k + 1, n):
            tmp = concat_A[i, k]
            for j in range(k, n + 1):
                concat_A[i, j] -= concat_A[k, j] * tmp

    # Backward
    X = np.empty([n, 1], dtype=np.float64)
    for i in reversed(range(n)):
        perm_index_i = x_permutations[i]
        X[perm_index_i] = concat_A[i, n]
        for j in range(i + 1, n):
            perm_index_j = x_permutations[j]
            X[perm_index_i] -= concat_A[i, j] * X[perm_index_j]

    return X

Решение $Ax = b$ методом Гаусса

In [10]:
X = gaussian_method(A, b)
X

array([[1.],
       [1.],
       [1.]])

Вывод $A * x$

In [11]:
A.dot(X)

array([[7.570463],
       [8.876384],
       [3.411906]])

Вывод $b$

In [12]:
b

array([[7.570463],
       [8.876384],
       [3.411906]])

Метод Гаусса

In [13]:
def jordan_method(A: np.ndarray, b: np.ndarray) -> np.ndarray:
    concat_A = np.append(A, b, axis=1)
    x_permutations = choose_main_element(concat_A)

    n = A.shape[0]
    # Forward
    for k in range(n):
        tmp = concat_A[k, k]
        if (tmp != 0):
            for j in range(k, n + 1):
                concat_A[k, j] /= tmp

        for i in range(n):
            if i == k:
                continue

            tmp = concat_A[i, k]
            for j in range(k, n + 1):
                concat_A[i, j] -= concat_A[k, j] * tmp

    # Backward
    X = np.empty([n, 1], dtype=np.float64)
    for i in reversed(range(n)):
        perm_index_i = x_permutations[i]
        X[perm_index_i] = concat_A[i, n]
        for j in range(i + 1, n):
            perm_index_j = x_permutations[j]
            X[perm_index_i] -= concat_A[i, j] * X[perm_index_j]

    return X

Построение обратной матрицы $A^{-1}$

In [14]:
ans = []
n = A.shape[0]
for i in range(n):
    temp_b = np.zeros([n, 1], dtype=np.float64)
    temp_b[i, 0] = 1
    ans.append(jordan_method(A, temp_b))

inverse_matrix = np.concatenate(ans, axis=1)
inverse_matrix

array([[ 0.13245943, -0.02766921,  0.07116938],
       [-0.02766921,  0.14851188, -0.03188242],
       [ 0.07116938, -0.03188242,  0.21812306]])

Проверка через умножение $A * A^{-1}$

In [15]:
I = A.dot(inverse_matrix)
I

array([[ 1.00000000e+00, -2.25218858e-17, -1.42236831e-16],
       [-2.11130520e-17,  1.00000000e+00,  2.23180183e-17],
       [ 2.58299402e-17, -1.75842952e-17,  1.00000000e+00]])

LU разложение

In [16]:
def LU_decomposition(A: np.ndarray):
    L = np.zeros(A.shape)
    U = np.zeros(A.shape)

    for i in range(n):
        for j in range(i, n):
            L[j, i] = A[j, i]
            for k in range(i):
                L[j, i] -= L[j, k] * U[k, i]

        for j in range(i, n):
            U[i, j] = A[i, j]
            for k in range(i):
                U[i, j] -= L[i, k] * U[k, j]
            U[i, j] /= L[i, i]
    return L, U

Вычисление определителя матрицы $L$ и $A$

In [17]:
L, U = LU_decomposition(A)

det_A = np.linalg.det(A)
det_L = 1.0
for i in range(n):
    det_L *= L[i, i]
if n == 0:
    det_L = 0.0

print(f"Det(A) = {det_A}")
print(f"Det(L) = {det_L}")

Det(A) = 297.3907770535562
Det(L) = 297.3907770535562
